# Experimenting for A1
- Figuring out how to solve the stuff

TODO: Refactor into a script

In [1]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List

In [4]:
# Helper functions
def to_gray(img): 
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.uint8)

def calc_color_hist(img, key=None):         
    return cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    
def create_norm_hist(img: np.ndarray, key=None) -> np.ndarray: 
    hist = calc_color_hist(img, key=key)
    return cv2.normalize(hist, hist).flatten()

def read_img(path: Path) -> np.ndarray: 
    return cv2.imread(str(path))

def process_img(img_path: Path):
    img = read_img(img_path)
    return create_norm_hist(img)

def compare_hists(source_hist, candidate_hist):
    return cv2.compareHist(source_hist, candidate_hist, cv2.HISTCMP_CHISQR)

In [2]:
DATA_DIR = Path("../../../CDS-VIS/flowers")
assert DATA_DIR.exists()

In [64]:
all_flowers = list(DATA_DIR.glob("*.jpg"))
test_flower = all_flowers[0]
all_flowers.remove(test_flower)
assert test_flower not in all_flowers

In [77]:
source_img = read_img(test_flower)
testy = create_norm_hist(source_img)

In [36]:
cv2.compareHist(testy, testy, cv2.HISTCMP_CHISQR)

0.0

In [46]:
source_hist = process_img(test_flower)



In [67]:
dist_df = pd.DataFrame({"dist": 0}, index=all_flowers)

In [68]:
dist_df

,dist
../../../CDS-VIS/flowers/image_0772.jpg,0
../../../CDS-VIS/flowers/image_1307.jpg,0
../../../CDS-VIS/flowers/image_0127.jpg,0
../../../CDS-VIS/flowers/image_0479.jpg,0
../../../CDS-VIS/flowers/image_1250.jpg,0
...,...
../../../CDS-VIS/flowers/image_0682.jpg,0
../../../CDS-VIS/flowers/image_1342.jpg,0
../../../CDS-VIS/flowers/image_1025.jpg,0
../../../CDS-VIS/flowers/image_1061.jpg,0


In [138]:
for i, flower in enumerate(all_flowers):
    candidate_hist = process_img(flower)
    dist_df.loc[flower, "dist"] += compare_hists(source_hist, candidate_hist)
    if i % 200 == 0: 
        print(f"processed {i} imgs!")

processed 0 imgs!
processed 200 imgs!
processed 400 imgs!
processed 600 imgs!
processed 800 imgs!
processed 1000 imgs!
processed 1200 imgs!


In [153]:
closest_imgs = dist_df.nsmallest(3, "dist")
output_dict = {"source": [test_flower.name], "1st": [closest_imgs.index[0].name], "2nd": [closest_imgs.index[1].name], "3rd": [closest_imgs.index[2].name]}
pd.DataFrame.from_dict(output_dict)

,source,1st,2nd,3rd
0,image_1079.jpg,image_0626.jpg,image_0228.jpg,image_0077.jpg


In [79]:
def add_text(img, text):
    new_img = img.copy()
    # setup text
    font = cv2.FONT_HERSHEY_SIMPLEX

    # get boundary of this text
    textsize = cv2.getTextSize(text, font, 1, 2)[0]

    # get coords based on boundary
    textX = (img.shape[1] - textsize[0]) // 2
    textY = (img.shape[0] + textsize[1]) // 2

    # add text centered on image
    cv2.putText(new_img, text, (textX, textY ), font, 1, (255, 255, 255), 2)
    
    return new_img

In [80]:
cv2.imwrite("test_img.jpg", add_text(source_img, "testy"))

True

In [82]:
def format_dist(dist: float) -> str:
    return f"dist:{dist: .2f}"

format_dist(2.132456)

'dist: 2.13'

In [132]:
def resize_square(img, size=300):
    return cv2.resize(img, dsize=(size, size))

In [133]:
dist_img_list = [resize_square(add_text(source_img, "SOURCE"))]
for filename, row in dist_df.nsmallest(3, "dist").iterrows():
    img = resize_square(read_img(filename))
    assert img.shape[1] == 300
    dist_img_list.append(add_text(img, format_dist(row["dist"])))

In [136]:
import math 
def arrange_square(img_list: List[np.ndarray], img_dim=300) -> np.ndarray: 
    """ Adapted from https://stackoverflow.com/a/52283965 """ 
    if len(img_list) not in [i**2 for i in range(10)]: 
        raise ValueError("List must have square number of elements")
    
    canvas_shape = math.isqrt(len(img_list))
    imgmatrix = np.zeros((canvas_shape*img_dim, canvas_shape*img_dim, 3), np.uint8)
    #Prepare an iterable with the right dimensions
    positions = itertools.product(range(canvas_shape), range(canvas_shape))

    for (y_i, x_i), img in zip(positions, img_list):
        x = x_i * img_dim
        y = y_i * img_dim
        imgmatrix[y:y+img_dim, x:x+img_dim, :] = img
    return imgmatrix
        

In [154]:
import itertools
testy = arrange_square(dist_img_list)

In [19]:
import itertools
import numpy as np

testy = {"a": np.array((1, 2)), "b": np.array((3, 2)), "c": np.array((1, 42)), "d": np.array((13, 2))}

wowow = list(itertools.combinations(testy.items(), 2))

In [3]:
from pathlib import PosixPath, Path
import pandas as pd

In [2]:
testy = [{'key': 'image_0772.jpg-image_1079.jpg', 'dist': 398.90088971033646}, {'key': 'image_1079.jpg-image_1307.jpg', 'dist': 308.6794332315007}, {'key': 'image_0127.jpg-image_1079.jpg', 'dist': 65.86221812570959}, {'key': 'image_0479.jpg-image_1079.jpg', 'dist': 1501.1858082961376}, {'key': 'image_1079.jpg-image_1250.jpg', 'dist': 427.9419007281442}]

In [4]:
full_df = pd.DataFrame(testy)


In [1]:
from typing import Tuple

In [4]:
def create_dist_row(arg: Tuple[Path, pd.DataFrame]) -> pd.DataFrame:
    source_path, dist_df = arg
    col_filter = (dist_df["path1"] == source_path.name) | (dist_df["path2"] == source_path.name)
    smallest_dists = dist_df[col_filter].nsmallest(3, "dist")
    smallest_dists["target"] = smallest_dists["key"].str.replace(source_path.name, "", regex=False).str.replace("-", "", regex=False)
    smallest_dists["source"] = source_path.name
    smallest_dists["rank"] = ["1st", "2nd", "3rd"]    
    return smallest_dists.pivot(index="source", columns = "rank", values="target")

In [5]:
test_df = pd.read_csv("output/all_dists.csv", index_col=0)

In [9]:
def combine_reversed_df(df):
    reversed_df = df.rename({"path1": "path2", "path2": "path1"}, axis=1)
    return pd.concat((df, reversed_df))

In [10]:
full_df = combine_reversed_df(test_df)

In [36]:
def find_smallest_cands(df, n=3, col="dist", target="path2"):
    return df.nsmallest(n, col)[[target]].assign(rank = ["1st", "2nd", "3rd"])

In [21]:
find_smallest_cands(full_df)

707555    ../../../CDS-VIS/flowers/image_0469.jpg
707555    ../../../CDS-VIS/flowers/image_0422.jpg
308221    ../../../CDS-VIS/flowers/image_0443.jpg
Name: path2, dtype: object

In [37]:
top_df = full_df.groupby("path1").apply(find_smallest_cands)

In [33]:
top_df.reset_index().pivot(index="path1", columns="rank", values="path2")

rank,1st,2nd,3rd
path1,,,
../../../CDS-VIS/flowers/image_0001.jpg,../../../CDS-VIS/flowers/image_0594.jpg,../../../CDS-VIS/flowers/image_0566.jpg,../../../CDS-VIS/flowers/image_0874.jpg
../../../CDS-VIS/flowers/image_0002.jpg,../../../CDS-VIS/flowers/image_0042.jpg,../../../CDS-VIS/flowers/image_0955.jpg,../../../CDS-VIS/flowers/image_1031.jpg
../../../CDS-VIS/flowers/image_0003.jpg,../../../CDS-VIS/flowers/image_0581.jpg,../../../CDS-VIS/flowers/image_1132.jpg,../../../CDS-VIS/flowers/image_0008.jpg
../../../CDS-VIS/flowers/image_0004.jpg,../../../CDS-VIS/flowers/image_0466.jpg,../../../CDS-VIS/flowers/image_0102.jpg,../../../CDS-VIS/flowers/image_1325.jpg
../../../CDS-VIS/flowers/image_0005.jpg,../../../CDS-VIS/flowers/image_1115.jpg,../../../CDS-VIS/flowers/image_1129.jpg,../../../CDS-VIS/flowers/image_1198.jpg
...,...,...,...
../../../CDS-VIS/flowers/image_1356.jpg,../../../CDS-VIS/flowers/image_0875.jpg,../../../CDS-VIS/flowers/image_0291.jpg,../../../CDS-VIS/flowers/image_0096.jpg
../../../CDS-VIS/flowers/image_1357.jpg,../../../CDS-VIS/flowers/image_1098.jpg,../../../CDS-VIS/flowers/image_0875.jpg,../../../CDS-VIS/flowers/image_1166.jpg
../../../CDS-VIS/flowers/image_1358.jpg,../../../CDS-VIS/flowers/image_0085.jpg,../../../CDS-VIS/flowers/image_1178.jpg,../../../CDS-VIS/flowers/image_0922.jpg


In [8]:
testy1 = test_df.head().rename({"path1": "path2", "path2": "path1"}, axis=1)
testy2 = test_df.head()
pd.concat([testy1, testy2])

,path2,path1,key,dist
0,../../../CDS-VIS/flowers/image_1079.jpg,../../../CDS-VIS/flowers/image_0772.jpg,image_0772.jpg-image_1079.jpg,398.900890
1,../../../CDS-VIS/flowers/image_1079.jpg,../../../CDS-VIS/flowers/image_1307.jpg,image_1079.jpg-image_1307.jpg,308.679433
2,../../../CDS-VIS/flowers/image_1079.jpg,../../../CDS-VIS/flowers/image_0127.jpg,image_0127.jpg-image_1079.jpg,65.862218
3,../../../CDS-VIS/flowers/image_1079.jpg,../../../CDS-VIS/flowers/image_0479.jpg,image_0479.jpg-image_1079.jpg,1501.185808
4,../../../CDS-VIS/flowers/image_1079.jpg,../../../CDS-VIS/flowers/image_1250.jpg,image_1079.jpg-image_1250.jpg,427.941901
0,../../../CDS-VIS/flowers/image_0772.jpg,../../../CDS-VIS/flowers/image_1079.jpg,image_0772.jpg-image_1079.jpg,398.900890
1,../../../CDS-VIS/flowers/image_1307.jpg,../../../CDS-VIS/flowers/image_1079.jpg,image_1079.jpg-image_1307.jpg,308.679433
2,../../../CDS-VIS/flowers/image_0127.jpg,../../../CDS-VIS/flowers/image_1079.jpg,image_0127.jpg-image_1079.jpg,65.862218
3,../../../CDS-VIS/flowers/image_0479.jpg,../../../CDS-VIS/flowers/image_1079.jpg,image_0479.jpg-image_1079.jpg,1501.185808
4,../../../CDS-VIS/flowers/image_1250.jpg,../../../CDS-VIS/flowers/image_1079.jpg,image_1079.jpg-image_1250.jpg,427.941901


In [15]:
test_df[["path1", "path2"]] = test_df["key"].str.split("-", expand=True)

In [20]:
testy1 = create_dist_row((test_path, test_df))
testy2 = create_dist_row((test_path, test_df))
pd.concat([testy1, testy2])

rank,1st,2nd,3rd
source,,,
image_0553.jpg,image_0233.jpg,image_1080.jpg,image_0234.jpg
image_0553.jpg,image_0233.jpg,image_1080.jpg,image_0234.jpg


In [9]:
test_path = Path("sample/image_0553.jpg")

In [7]:
%load_ext line_profiler

In [26]:
%lprun -f create_dist_row create_dist_row((test_path, test_df))

Timer unit: 1e-06 s

Total time: 0.150693 s
File: /tmp/ipykernel_791/2516022608.py
Function: create_dist_row at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_dist_row(arg: Tuple[Path, pd.DataFrame]) -> pd.DataFrame:
     2         1          4.0      4.0      0.0      source_path, dist_df = arg
     3         1     143349.0 143349.0     95.1      col_filter = (dist_df["path1"] == source_path.name) | (dist_df["path2"] == source_path.name)
     4         1       2409.0   2409.0      1.6      smallest_dists = dist_df[col_filter].nsmallest(3, "dist")
     5         1       1593.0   1593.0      1.1      smallest_dists["target"] = smallest_dists["key"].str.replace(source_path.name, "", regex=False).str.replace("-", "", regex=False)
     6         1        442.0    442.0      0.3      smallest_dists["source"] = source_path.name
     7         1        626.0    626.0      0.4      smallest_dists["rank"] = ["1s